# Extract MODIS Site Data and Generate Samples
## MODIS Site Data
For each site in the LFMC sample data, extract the full time series of MODIS reflectance and snow-cover data, and save to CSV files. Note: if the output csv files already exist they are assumed to be correct and are not over-written. The data is gap-filled after being saved but before being used to create the MODIS sample data.
## MODIS Sample Data
For each sample, extract the timeseries MODIS reflectance data. The timeseries length is determined by the MODIS_TS_LENGTH value. The sample is rejected if the full timeseries cannot be extracted (start/end outside the full site time series). The extracted MODIS data is combined to a single dataframe and saved. Another LFMC sample dataset containing only the valid samples is also created.
## Input Files
- `LFMC_sites.csv` and `LFMC_samples.csv` created by the `Extract Auxiliary Data.ipynb` notebook.

## Output Files
- The extracted sites data are created in `MODIS_DIR` (by default `MCD43A4` located in `LFMC_DATA_DIR/GEE_DIR`). The directory will contain a CSV file for each site.
- The extracted MODIS data for each samples and updated samples data are created in `DATASETS_DIR`. File names include the time series length (i.e. 730days) of the extracted MODIS data. So, with the default settings they are `modis_730days.csv` and `samples_730days.csv`.

## Notes
1. This notebook should be run after running the `Extract Auxiliary Data.ipynb` notebook.
2. The generated name for `GEE_DIR` includes the projection and scale of the extracted GEE data. 
3. It will take about 8.5 hours to run if there are no existing site extracts.
4. 2364 samples will not be extracted as they were collected before March 2002 (so less than 2 years MODIS data available).
5. There should be no invalid sites, but occasionally extraction from GEE will fail for a site. If this happens re-run the notebook (keep the existing site CSV files so they are not re-extracted).

In [1]:
import os
os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import get_sample_data, sort_key
from timeseries_extractor import GeeTimeseriesExtractor

### Program parameters and constants

GEE Parameters
- Reflectance product is MCD43A4 - daily reflectance using 8-day composites
- Scale set to use native MODIS resolution
- Extracts two years' of data for each sample to allow for a 1-year time series with a 1-year lead time

In [2]:
START_DATE = "2000-03-01"
END_DATE = "2019-01-01"  # Final day retrieved will be 2018-12-31

# MODIS time series constants
MODIS_TS_LENGTH = 730
MODIS_TS_OFFSET = 1
MODIS_TS_FREQ = 1

# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]
# Extract data using default MODIS scale and projection
SCALE = 463.3127
PROJ = "SR-ORG:6974"

EARLIEST_SAMPLE = datetime.strptime(START_DATE, '%Y-%m-%d') + timedelta(
    days=MODIS_TS_LENGTH * MODIS_TS_FREQ + MODIS_TS_OFFSET)

### Directories and Files

In [3]:
# Sub-directories for GEE extracts
GEE_DIR = f"GEE_{common.PROJ.replace(':', '-')}_{int(common.SCALE)}"
MODIS_DIR = os.path.join(common.LFMC_DATA_DIR, GEE_DIR, "MCD43A4")

# File Names
SAMPLES_OUTPUT = os.path.join(common.DATASETS_DIR, f"samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")
MODIS_OUTPUT = os.path.join(common.DATASETS_DIR, f"modis_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")

# Create output directories if necessary
if not os.path.exists(MODIS_DIR):
    os.makedirs(MODIS_DIR)

## Main Processing
Connect to GEE

In [4]:
import ee
ee.Initialize()

### Generate MODIS sample data

For each site, get the sample data for each sample at the site

Note: This gets the data using the default MODIS scale and projection. To change the scale and/or projection, add calls to the `GeeTimeseriesExtractor.setProjScale` method

In [5]:
sites = pd.read_csv(common.LFMC_SITES, float_precision="high")
samples = pd.read_csv(common.LFMC_SAMPLES, float_precision="high")
modis_extractor = GeeTimeseriesExtractor(PRODUCT, BANDS, START_DATE, END_DATE,
                                         dir_name=MODIS_DIR)
modis_data = []
valid_data = [False] * samples.shape[0]
invalid_pixels = []
invalid_sites = []
for site_idx, site in sites.iterrows():
    print(f'Processing site {site.Site}')
    site_samples = samples[samples.Site == site.Site]
    try:
        modis_df = modis_extractor.get_and_save_data(site)
    except:
        print(f'Failed to extract data for {site.Site}')
        invalid_sites.append(site.Site)
        continue
    for index, sample in site_samples.iterrows():
        sample_data = get_sample_data(sample["Sampling date"], modis_df,
                                      MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)
        if sample_data is None or np.isnan(sample_data.sum()):
            invalid_pixels.append(index)
        else:
            modis_data.append([sample.ID] + list(sample_data))
            valid_data[index] = True

Processing site C4_1
Processing site C4_2
Processing site C4_3
Processing site C4_4
Processing site C4_5
Processing site C6_1
Processing site C6_2
Processing site C6_3
Processing site C6_4
Processing site C6_5
Processing site C6_6
Processing site C6_7
Processing site C6_8
Processing site C6_9
Processing site C6_10
Processing site C6_11
Invalid date: 2000-04-07 outside valid date range
Invalid date: 2000-04-21 outside valid date range
Invalid date: 2000-05-05 outside valid date range
Invalid date: 2000-05-19 outside valid date range
Invalid date: 2000-06-02 outside valid date range
Invalid date: 2000-06-16 outside valid date range
Invalid date: 2000-06-30 outside valid date range
Invalid date: 2000-07-14 outside valid date range
Invalid date: 2000-07-28 outside valid date range
Invalid date: 2001-04-07 outside valid date range
Invalid date: 2001-04-21 outside valid date range
Invalid date: 2001-05-05 outside valid date range
Invalid date: 2001-05-19 outside valid date range
Invalid date

Summary of sites/samples not extracted

In [6]:
print(f'Invalid sites: {len(invalid_sites)}; Invalid pixels: {len(invalid_pixels)}; ')
print(invalid_sites)
print(invalid_pixels)

Invalid sites: 0; Invalid pixels: 2364; 
[]
[1472, 1473, 1474, 1475, 1476, 1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 6253, 6254, 6255, 6256, 6257, 6258, 6259, 6260, 6261, 6262, 6263, 6264, 6265, 6266, 6267, 6268, 6269, 6270, 6271, 6272, 6273, 14025, 14026, 14027, 14028, 14029, 14030, 14031, 14032, 14033, 14034, 14035, 14036, 14037, 14038, 14039, 14040, 14041, 14042, 14043, 14044, 14045, 14046, 14047, 14048, 16481, 16486, 16491, 16496, 16502, 16509, 16515, 16554, 16565, 16576, 16587, 16597, 16603, 16609, 22595, 22596, 22597, 22598, 22599, 22600, 22601, 22602, 22603, 22604, 22605, 22606, 22607, 22608, 22609, 22610, 22611, 22612, 22613, 22614, 22615, 28366, 28367, 28368, 28369, 28370, 28371, 28372, 28373, 28374, 28375, 28376, 28377, 28378, 28379, 28380, 28381, 28382, 28499, 28500, 28501, 28502, 28503, 28504, 28505, 28506, 28507, 35452, 35453, 35465, 35476, 35484, 35496, 35507, 35519, 35530, 35541, 35552, 35563, 35568, 35575

### Save Results
Save and display sample reflectance data

In [7]:
modis_data = pd.DataFrame(modis_data)
ts_days = (MODIS_TS_LENGTH - 1) * MODIS_TS_FREQ
modis_data.columns = ["ID"] + [f'{day-MODIS_TS_OFFSET:04}_{band+1}'
                               for day in range(-ts_days, 1, MODIS_TS_FREQ)
                               for band in range(len(BANDS))]
modis_data.sort_values('ID', inplace=True, key=lambda x: x.apply(sort_key))
modis_data.to_csv(MODIS_OUTPUT, index=False)
modis_data

,ID,-730_1,-730_2,-730_3,-730_4,-730_5,-730_6,-730_7,-729_1,-729_2,...,-002_5,-002_6,-002_7,-001_1,-001_2,-001_3,-001_4,-001_5,-001_6,-001_7
0,C4_1_1,2099,2834,1239,1714,3521,3801,3363,2098,2827,...,3285,3182,2553,1701,2738,900,1408,3285,3203,2568
1,C4_1_2,2099,2810,1240,1707,3431,3786,3333,2097,2806,...,3517,3526,2816,1820,2847,1015,1488,3506,3523,2816
2,C4_1_3,2121,2865,1237,1722,3493,3785,3254,2117,2862,...,3439,3521,2841,1896,2833,1036,1504,3452,3528,2847
3,C4_1_4,1868,2530,1087,1499,3086,3578,3048,1875,2542,...,3436,3480,2765,1849,2728,1039,1472,3404,3462,2754
4,C4_1_5,1890,2584,1125,1541,3258,3630,3118,1886,2575,...,3297,3368,2683,1815,2702,976,1420,3287,3326,2642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66398,C13_4_14,510,1850,277,497,2009,1585,762,502,1836,...,2358,1922,981,652,2062,351,589,2421,1931,1005
66399,C13_4_15,483,1792,267,465,1907,1565,728,498,1800,...,2383,1840,948,625,2039,342,574,2369,1811,925
66400,C13_4_16,500,1758,270,475,1938,1474,725,501,1742,...,2294,1798,960,640,1977,345,562,2294,1805,961
66401,C13_4_17,496,1736,271,463,1823,1350,686,503,1755,...,2208,1492,830,601,1924,320,524,2197,1489,824


Save and display the valid samples

In [8]:
valid_samples = samples[valid_data].sort_values('ID', key=lambda x: x.apply(sort_key))
valid_samples.to_csv(SAMPLES_OUTPUT, index=False)
valid_samples

,ID,Latitude,Longitude,Sampling date,Sampling year,Land Cover,LFMC value,Site,Czone1,Czone2,Czone3,Day_sin,Day_cos,Long_sin,Long_cos,Lat_norm,Elevation,Slope,Aspect_sin,Aspect_cos
163,C4_1_1,40.21458,-112.21868,2005-06-20,2005,Shrubland,156.76300,C4_1,B,BS,BSk,-0.21352,0.97694,-0.92575,-0.37814,0.72341,0.26207,0.01972,-0.03023,0.99954
164,C4_1_2,40.21458,-112.21868,2005-07-05,2005,Shrubland,128.27700,C4_1,B,BS,BSk,0.04302,0.99907,-0.92575,-0.37814,0.72341,0.26207,0.01972,-0.03023,0.99954
165,C4_1_3,40.21458,-112.21868,2005-07-21,2005,Shrubland,92.48200,C4_1,B,BS,BSk,0.31311,0.94972,-0.92575,-0.37814,0.72341,0.26207,0.01972,-0.03023,0.99954
166,C4_1_4,40.21458,-112.21868,2005-08-08,2005,Shrubland,82.09300,C4_1,B,BS,BSk,0.58779,0.80902,-0.92575,-0.37814,0.72341,0.26207,0.01972,-0.03023,0.99954
167,C4_1_5,40.21458,-112.21868,2005-08-23,2005,Shrubland,78.95300,C4_1,B,BS,BSk,0.77488,0.63210,-0.92575,-0.37814,0.72341,0.26207,0.01972,-0.03023,0.99954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,C13_4_14,46.89791,-113.43535,2012-08-28,2012,"Tree cover, needleleaved, evergreen, closed (>...",102.44207,C13_4,B,BS,BSk,0.83593,0.54884,-0.91751,-0.39771,0.76054,0.21077,0.07241,0.53863,0.84254
61,C13_4_15,46.89791,-113.43535,2012-09-04,2012,"Tree cover, needleleaved, evergreen, closed (>...",88.76436,C13_4,B,BS,BSk,0.89584,0.44438,-0.91751,-0.39771,0.76054,0.21077,0.07241,0.53863,0.84254
62,C13_4_16,46.89791,-113.43535,2012-09-11,2012,"Tree cover, needleleaved, evergreen, closed (>...",88.79382,C13_4,B,BS,BSk,0.94276,0.33347,-0.91751,-0.39771,0.76054,0.21077,0.07241,0.53863,0.84254
63,C13_4_17,46.89791,-113.43535,2012-09-18,2012,"Tree cover, needleleaved, evergreen, closed (>...",81.72345,C13_4,B,BS,BSk,0.97601,0.21772,-0.91751,-0.39771,0.76054,0.21077,0.07241,0.53863,0.84254
